In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import clear_output

import os
from selenium.webdriver.support import expected_conditions

import json
from sc_header import createDriver, agregar_links, scrape_url, scrape_batch,aplanar_lista, check_length

In [2]:

browser = createDriver()

In [3]:
#browser.delete_all_cookies()

In [4]:

fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')


In [5]:
links_sin_color = [['https://www.shein.com/T-Shirts-c-1738.html?icn=t-shirts&ici=www_tab01navbar04menu11dir01',
                    'REMERA'],
                   ['https://www.shein.com/Tank-Tops-Camis-c-1779.html?icn=tank-tops-camis&ici=www_tab01navbar04menu11dir03',
                    'CAMISETAS S/MANGA'],
                   ['https://www.shein.com/Bodysuits-c-1882.html?icn=bodysuits&ici=www_tab01navbar04menu11dir05',
                   'ENTERITO']
                    ]

In [6]:
url_base = 'https://www.shein.com'
Tlistas = []
for lsc in links_sin_color:
    
    agregar_links(lsc[0],
                 browser,
                 Tlistas,
                 lsc[1],
                 url_base)
    

In [7]:
set([item[0] for item in Tlistas])

{'CAMISETAS S/MANGA', 'ENTERITO', 'REMERA'}

In [8]:
set([item[1] for item in Tlistas])

{'Army Green',
 'Beige',
 'Black',
 'Black and White',
 'Blue',
 'Bright',
 'Brown',
 'Burgundy',
 'Camel',
 'Gold',
 'Green',
 'Grey',
 'Khaki',
 'Multicolor',
 'Navy',
 'Nude',
 'Orange',
 'Pastel',
 'Pink',
 'Purple',
 'Red',
 'White',
 'Yellow'}

In [9]:
#Tlistas_copy = [i for i in Tlistas if i[0] not in ["REMERA"]]

In [10]:
browser = createDriver()
hrefs_list = []
ex_col = []
for lista in Tlistas:
    try:
        browser.get(lista[2])
    except:
        browser.quit()
        time.sleep(1)
        browser = createDriver()
        browser.get(lista[2])
    
    while True:
        soup = BeautifulSoup(browser.page_source,'html.parser')
        
        indice_=0
        for element in soup.select('div.j-switch-color-wrap > div.c-goodsitem__ratiowrap > a'):
        #for element in soup.find_all(class_='c-goodsitem__ratiowrap'):
            
            try:
                
                href_aux = url_base +'/'+ element['href']
                
                hrefs_list.append([indice_,
                                   lista[0],
                                   lista[1],
                                   lista[2],
                                   href_aux
                                  ])
                indice_+=1
                
            except:
                ex_col.append([lista[2],lista[1]])
                pass            
        try:
            browser.find_element_by_class_name('page-next').find_element_by_tag_name('a').click()
        except:
            break


In [11]:
browser.quit()

In [12]:
df = pd.DataFrame(hrefs_list,columns=['pos','tipo','color','url_scraper','href'])


df['auxi'] = df['href'].apply(check_length)

#df['auxi'] = df['href'].apply(lambda x: str(max(map(int,re.findall(r"([\d]+)",x)))) )

In [13]:
if len(df['tipo'].unique().tolist()) != len(set([item[0] for item in Tlistas])):
    print('REVISAR LOS TIPOS SCRAPEADOS')
    raise

In [14]:
df['tipo'].unique().tolist()

['REMERA', 'CAMISETAS S/MANGA', 'ENTERITO']

In [15]:
len(df.drop_duplicates()),len(df)

(6381, 6381)

In [16]:
start_ = datetime.datetime.now()
items = []
errores = []

batch_size = 5
url_chunks = [df[x:x+batch_size] for x in range(0, len(df), batch_size)]

var = 0
for ix, url_chunk in enumerate(url_chunks):
    scrape_batch(errores,items,url_chunk)
    
    if int(ix/50) == ix/50:
        
        try:
            df = pd.DataFrame(items)
            df.to_excel('remeras.xlsx')
            
            if len(errores) >0:
                
                pd.DataFrame(errores).to_excel('remeras_error.xlsx')
                
            file = open("remeras.txt", "w")
            file.write( str(url_chunk) + os.linesep)
            file.close()
        except:
            print("Erro al crear .xlsx")    
    
end_ = datetime.datetime.now()

In [17]:
# start_ = datetime.datetime.now()
# items = []

# batch_size = 5
# url_chunks = [df[x:x+batch_size] for x in range(0, len(df), batch_size)]

# var = 0
# for url_chunk in url_chunks:
#     items.append(scrape_batch(url_chunk))
#     var += 1
    
#     if var == 50:
#         var = 0
#         try:
#             df = pd.DataFrame(items)
#             df.to_excel('remeras.xlsx')
            
                                                                    
#             file = open("remeras.txt", "w")
#             file.write( str(url_chunk) + os.linesep)
#             file.close()
#         except:
#             print("Erro al crear .xlsx")
        
    
# end_ = datetime.datetime.now()

In [18]:
print('Tiempo de ejecución: {}'.format(end_ - start_)[:-4])

Tiempo de ejecución: 9:14:29.12


In [19]:
len(items)

26822

In [20]:
new_list = aplanar_lista(items)
new_list = [i for i in new_list if type(i) != int]
dfShein = pd.DataFrame(new_list)

In [21]:
len(dfShein)

26822

In [22]:
dfShein = dfShein.rename(columns={0: 'pos'})
dfShein = dfShein.rename(columns={1: 'Tipo'})
dfShein = dfShein.rename(columns={2: 'Color'})
dfShein = dfShein.rename(columns={3: 'id Producto'})
dfShein = dfShein.rename(columns={4: 'Descripcion'})
dfShein = dfShein.rename(columns={5: 'Talle'})
dfShein = dfShein.rename(columns={6: 'Stock'})
dfShein = dfShein.rename(columns={7: 'Precio'})
dfShein = dfShein.rename(columns={8: 'Precio_dto'})
dfShein = dfShein.rename(columns={9: 'Url Imagen'})
dfShein = dfShein.rename(columns={10: 'Url Producto'})
dfShein = dfShein.rename(columns={11: 'pagina_scraper'})

In [23]:
dfShein["Fecha"] = hoy
dfShein["Marca"] = "SHEIN"
dfShein["Moneda"] = "DOLAR US$"
dfShein['Sexo'] = 'Mujer'
dfShein['Descripcion aux'] = '0'

In [24]:


paginas = len(dfShein['pagina_scraper'].unique())
file = open("remeras.txt", "a")
file.write(  os.linesep +f'CANTIDAD_ITEMS - {len(dfShein)},'+ f' DURACION - {format(end_ - start_)[:-4]},'+
           f' CANTIDAD_HOJAS - {paginas},'+ f' FECHA_ALTA - {hoy}' + os.linesep   
          )
file.close()

In [25]:
if len(dfShein['Tipo'].unique().tolist()) != len(set([item[0] for item in Tlistas])):
    print('REVISAR LOS TIPOS SCRAPEADOS')
    raise

In [26]:
dfShein.to_excel(f'../Salida/dfShein_remeras{fecha}.xlsx')

In [27]:
len(dfShein.drop_duplicates()),len(dfShein.dropna()),len(dfShein)

(26822, 26822, 26822)

In [28]:
#restore = pd.read_excel('remeras.xlsx')

In [29]:
# #COPIADO
# startt = datetime.datetime.now()
# var = 0
# for url_chunk in chunks_copy:
#     items.append(scrape_batch(url_chunk))
#     var += 1
    
#     if var == 50:
#         var = 0
#         try:
#             df = pd.DataFrame(items)
#             df.to_excel('remeras.xlsx')
            
                                                                    
#             file = open("remeras.txt", "w")
#             file.write( str(url_chunk) + os.linesep)
#             file.close()
#         except:
#             print("Erro al crear .xlsx")
        
    
# end_ = datetime.datetime.now()